In [188]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_datareader.famafrench import get_available_datasets,FamaFrenchReader
import pandas_datareader.data as web
import warnings
import statsmodels.api as sm


warnings.simplefilter(action="ignore", category=FutureWarning)

tickers = [
    'DIS', 'UHS', 'GOOGL', 'LOW', 'BSX', 'JPM', 'EOG', 'SBAC', 'STT', 'DVN',
    'VRTX', 'BK', 'ALL', 'WY', 'OXY', 'ON', 'EXC', 'UAL', 'AKAM', 'SMCI',
    'LYV', 'AMD', 'VLO', 'NI', 'DRI', 'SJM', 'APH', 'MAS', 'CMG', 'TXT'
]

n_tickers = len(tickers)

start_date = '2014-01-01'
end_date = '2023-12-31'

# Download the daily data from stocks
price = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
monthly_returns_stocks = price.resample("M").ffill().pct_change()
monthly_returns_stocks.index = monthly_returns_stocks.index.to_period('M')
monthly_returns_stocks = monthly_returns_stocks.dropna()

# Download factors. All data is monthly, so it is necessary to change the returns to be monthly returns
research_factors = FamaFrenchReader("F-F_Research_Data_Factors",start=pd.to_datetime(start_date),end=pd.to_datetime(end_date)).read()[0]
industry_factors = FamaFrenchReader("5_Industry_Portfolios",start=pd.to_datetime(start_date),end=pd.to_datetime(end_date)).read()[0] # Average Value Weighted Returns -- Monthly
momentum_factor = FamaFrenchReader("F-F_Momentum_Factor",start=pd.to_datetime(start_date),end=pd.to_datetime(end_date)).read()[0]

# Merge all factors
factors = pd.concat([research_factors,industry_factors,momentum_factor],axis=1)
factors = factors.drop(columns=["RF"])
factors = factors.loc[:].div(100)
factors = factors.iloc[1:,:]

In [191]:
def fit_factor_model(returns, factors):
    X = sm.add_constant(factors)
    result = sm.OLS(returns, X).fit()
    return result.params

betas = pd.DataFrame(index=monthly_returns_stocks.columns, columns = factors.columns)
for stock in monthly_returns_stocks.columns:
    betas.loc[stock] = fit_factor_model(monthly_returns_stocks[stock], factors)
    
betas

,Mkt-RF,SMB,HML,Cnsmr,Manuf,HiTec,Hlth,Other,Mom
Ticker,,,,,,,,,
AKAM,0.927721,-0.330877,-0.067711,0.239232,-0.286513,0.095483,-0.389753,0.163335,0.2724
ALL,-2.147798,-0.327829,-0.000937,0.33476,0.560779,0.367792,0.428213,1.186806,0.152053
AMD,13.243411,-0.255783,0.404932,-2.524633,-2.832053,-2.370435,-0.363657,-3.17456,-0.249053
APH,-1.027417,0.151267,0.284342,0.544177,0.514791,0.858325,0.218784,0.030233,-0.07132
BK,1.979075,-0.292185,0.446767,-0.538502,-0.792377,-0.398617,0.015925,0.639179,-0.298419
BSX,-3.586107,-0.065589,-0.078029,0.560972,0.953118,1.214512,1.064386,0.786881,0.061669
CMG,-5.137929,0.245059,-0.159296,2.02656,0.662264,2.015484,0.760273,0.828858,-0.033383
DIS,1.681131,-0.20559,-0.778854,-0.411226,-0.087968,-0.795041,-0.506841,1.169948,-0.20592
DRI,-2.113455,0.107976,-0.265199,1.122194,0.317426,0.198553,0.006353,1.432013,0.070434


In [193]:
factor_cov_matrix = factors.cov()
factor_cov_matrix

,Mkt-RF,SMB,HML,Cnsmr,Manuf,HiTec,Hlth,Other,Mom
Mkt-RF,0.002074,0.000386,0.000043,0.001983,0.001953,0.002200,0.001569,0.002210,-0.000747
SMB,0.000386,0.000760,0.000064,0.000317,0.000382,0.000302,0.000343,0.000500,-0.000331
HML,0.000043,0.000064,0.001401,-0.000218,0.000638,-0.000380,-0.000215,0.000598,-0.000442
Cnsmr,0.001983,0.000317,-0.000218,0.002167,0.001727,0.002137,0.001444,0.001964,-0.000624
Manuf,0.001953,0.000382,0.000638,0.001727,0.002349,0.001818,0.001420,0.002250,-0.000930
HiTec,0.002200,0.000302,-0.000380,0.002137,0.001818,0.002647,0.001551,0.002114,-0.000697
Hlth,0.001569,0.000343,-0.000215,0.001444,0.001420,0.001551,0.001884,0.001593,-0.000288
Other,0.002210,0.000500,0.000598,0.001964,0.002250,0.002114,0.001593,0.002762,-0.001005
Mom,-0.000747,-0.000331,-0.000442,-0.000624,-0.000930,-0.000697,-0.000288,-0.001005,0.001572


In [215]:
specific_variances = pd.Series(index=monthly_returns_stocks.columns)

for stock in monthly_returns_stocks.columns:
    residuals = monthly_returns_stocks[stock] - factors.dot(betas.loc[stock])
    specific_variances[stock] = residuals.var()
    
estimate_with_factors_cov_matrix = betas.dot(factor_cov_matrix).dot(betas.T) + pd.DataFrame(np.diag(specific_variances), index=monthly_returns_stocks.columns, columns=monthly_returns_stocks.columns)

In [216]:
estimate_with_factors_cov_matrix

Ticker,AKAM,ALL,AMD,APH,BK,BSX,CMG,DIS,DRI,DVN,EOG,EXC,GOOGL,JPM,LOW,LYV,MAS,NI,ON,OXY,SBAC,SJM,SMCI,STT,TXT,UAL,UHS,VLO,VRTX,WY
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AKAM,0.007118,0.000723,0.003185,0.001591,0.00132,0.000916,0.001832,0.001799,0.001476,0.001994,0.00107,0.000631,0.001823,0.001369,0.001708,0.001778,0.001856,0.000592,0.002476,0.001091,0.001107,0.000437,0.001789,0.001783,0.001636,0.001671,0.001325,0.001499,0.000715,0.001927
ALL,0.000723,0.003216,0.001727,0.001325,0.0017,0.001101,0.000829,0.00164,0.001363,0.003269,0.002347,0.000704,0.000849,0.001947,0.001304,0.001474,0.001562,0.000608,0.001792,0.002157,0.000648,0.000362,0.000632,0.002147,0.002036,0.002161,0.002119,0.002389,0.00097,0.00191
AMD,0.003185,0.001727,0.026709,0.004652,0.003929,0.003133,0.005044,0.004845,0.003545,0.007409,0.003791,0.001247,0.00499,0.003982,0.00451,0.00484,0.005024,0.001032,0.007876,0.004362,0.002762,0.000984,0.004578,0.005609,0.004932,0.003954,0.003875,0.00414,0.003355,0.0053
APH,0.001591,0.001325,0.004652,0.003896,0.002508,0.001754,0.002518,0.002863,0.002315,0.005946,0.003774,0.001167,0.002333,0.002615,0.002715,0.002894,0.003001,0.001016,0.004716,0.004076,0.001247,0.000799,0.002669,0.003464,0.003318,0.003347,0.002691,0.003592,0.001437,0.003619
BK,0.00132,0.0017,0.003929,0.002508,0.004761,0.001543,0.00162,0.00317,0.002405,0.00606,0.004071,0.000791,0.001916,0.003657,0.002072,0.002983,0.002595,0.00064,0.004233,0.00397,0.00062,0.000454,0.00168,0.004245,0.003527,0.004273,0.003067,0.004003,0.001321,0.00331
BSX,0.000916,0.001101,0.003133,0.001754,0.001543,0.00387,0.001689,0.001761,0.001429,0.003447,0.002175,0.000747,0.00143,0.001648,0.001844,0.001764,0.002085,0.000705,0.002501,0.002219,0.001249,0.000529,0.001234,0.0022,0.002222,0.001721,0.002271,0.002281,0.00185,0.002284
CMG,0.001832,0.000829,0.005044,0.002518,0.00162,0.001689,0.00972,0.002509,0.002183,0.00313,0.001493,0.000722,0.00262,0.001455,0.002826,0.002678,0.003004,0.000787,0.003961,0.001867,0.001857,0.000887,0.002525,0.002547,0.002467,0.002212,0.001988,0.00209,0.001994,0.003104
DIS,0.001799,0.00164,0.004845,0.002863,0.00317,0.001761,0.002509,0.006461,0.002869,0.005742,0.003679,0.000937,0.002406,0.003382,0.002638,0.003653,0.003091,0.000725,0.004631,0.003646,0.001251,0.00056,0.002541,0.004192,0.003948,0.004266,0.003083,0.003932,0.001746,0.003654
DRI,0.001476,0.001363,0.003545,0.002315,0.002405,0.001429,0.002183,0.002869,0.007276,0.004591,0.002909,0.000834,0.001774,0.002602,0.002431,0.002764,0.002669,0.000702,0.003589,0.002994,0.000995,0.000653,0.001768,0.003243,0.003136,0.003537,0.002574,0.00322,0.001251,0.003193


In [217]:
regular_matrix_cov = monthly_returns_stocks.cov()
regular_matrix_cov

Ticker,AKAM,ALL,AMD,APH,BK,BSX,CMG,DIS,DRI,DVN,EOG,EXC,GOOGL,JPM,LOW,LYV,MAS,NI,ON,OXY,SBAC,SJM,SMCI,STT,TXT,UAL,UHS,VLO,VRTX,WY
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AKAM,0.007118,0.000897,0.002199,0.001495,0.001246,0.000245,0.002360,0.001955,0.001027,0.001180,0.000908,0.000614,0.001549,0.001413,0.001678,0.001174,0.001662,0.000274,0.001941,0.000940,0.001212,0.001082,0.002383,0.001506,0.001595,0.001025,0.001866,0.001325,0.001176,0.001266
ALL,0.000897,0.003216,0.000302,0.001118,0.001754,0.000745,0.001181,0.001796,0.001580,0.003872,0.002329,0.001129,0.000646,0.001814,0.000896,0.000818,0.001378,0.001156,0.001205,0.002872,0.000793,0.000296,0.000277,0.002009,0.002111,0.002111,0.001953,0.002725,0.000630,0.001698
AMD,0.002199,0.000302,0.026709,0.004531,0.003739,0.003747,0.003098,0.003749,0.002313,0.007054,0.003321,0.001121,0.004715,0.004490,0.004970,0.005061,0.004753,0.001083,0.009130,0.003026,0.002374,0.000655,0.008282,0.005474,0.004851,0.002793,0.004209,0.004177,0.002698,0.006009
APH,0.001495,0.001118,0.004531,0.003896,0.002613,0.001657,0.002442,0.002644,0.002715,0.005504,0.003122,0.001133,0.002092,0.002579,0.002794,0.002938,0.003144,0.000961,0.005344,0.003082,0.001157,0.000616,0.003054,0.003607,0.003425,0.003565,0.002887,0.003865,0.000981,0.003800
BK,0.001246,0.001754,0.003739,0.002613,0.004761,0.001456,0.001615,0.002969,0.001984,0.006039,0.003983,0.000747,0.001811,0.003706,0.002188,0.002605,0.002990,0.000753,0.004266,0.004191,0.000627,0.000236,0.001354,0.005218,0.003350,0.003709,0.003120,0.004015,0.001519,0.003243
BSX,0.000245,0.000745,0.003747,0.001657,0.001456,0.003870,0.001956,0.002135,0.001762,0.003734,0.002495,0.000535,0.001435,0.001493,0.001903,0.002084,0.001772,0.000719,0.002246,0.001836,0.001250,0.000453,0.000609,0.002000,0.002196,0.001403,0.002714,0.002561,0.001415,0.002806
CMG,0.002360,0.001181,0.003098,0.002442,0.001615,0.001956,0.009720,0.003082,0.003801,0.004471,0.002048,0.000621,0.003051,0.001122,0.002762,0.002020,0.003379,0.000724,0.002857,0.002769,0.001885,0.000245,0.002287,0.002592,0.001854,0.002758,0.003159,0.001864,0.001734,0.003297
DIS,0.001955,0.001796,0.003749,0.002644,0.002969,0.002135,0.003082,0.006461,0.003279,0.006405,0.004026,0.000749,0.002344,0.003600,0.002481,0.004037,0.002743,0.000518,0.003847,0.003813,0.001233,0.000609,0.000406,0.003820,0.003640,0.004267,0.003316,0.004073,0.001710,0.003239
DRI,0.001027,0.001580,0.002313,0.002715,0.001984,0.001762,0.003801,0.003279,0.007276,0.006512,0.003360,0.000627,0.001946,0.002410,0.002700,0.003443,0.002213,0.000585,0.003340,0.004502,0.000450,0.000137,0.002916,0.002595,0.003148,0.004889,0.002713,0.004154,0.000819,0.003631


Nao entendi direto o ultimo ponto:

- Compare the performance of the MVP (minimum variance portfolio) using the regular matrix and the factor-based one in the full sample!
    Com a matriz de covariancia, consigo encontrar o desvio padrao porem o expected return 'e o mesmo. O que isso quer dizer? Factor model 'e so uma maneira diferente de encontrar o risco

- Para calcular 